# Nose poke analysis

Brief 1-2 sentence description of notebook.

In [31]:
import os
import glob
import git
import sys


In [32]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
# import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import h5py
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter


In [33]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [34]:
git_root

'C:/Users/yongc/Documents/GitHub/nose_poke_identifier'

In [35]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [36]:
import utilities.helper
import sleap.process_pose

In [37]:
# sns.set('notebook', 'ticks', font_scale=1.2)
mpl.rcParams['figure.figsize'] = [15,6]

## Inputs & Data

Explanation of each input and where it comes from.

In [38]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
NOSE_POKE_DF = pd.read_pickle(r"./proc/rce_pilot_2_nose_pokes.pkl")


OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs
MED_PC_WIDTH = 29.5
MED_PC_HEIGHT = 24
FRAME_RATE = 22
WINDOW_SIZE = 25
DISTANCE_THRESHOLD = 2

In [39]:
START_STOP_FRAME_DF = pd.read_excel("./rce_per_subject_start_stop_video_frame.xlsx").dropna(subset=["file_path"])

In [40]:
NOSE_POKE_DF.head()

,session_dir,tone_frames,box_1_port_entry_frames,video_name,session_path,all_subjects,first_timestamp,last_timestamp,tone_timestamps,box_1_port_entry_timestamps,...,nose_index,entry_length,box_1_port_entry_start,box_1_port_entry_end,sliced_locations,nose_coordinates,nose_to_reward_port_distance,nose_to_reward_port_distance_x,entry_winner,contested
0,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[192, 248]",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,"[1.1, 1.2]",7977066,76318450,"[[1126742, 1326741], [3526740, 3726740], [5826...","[192745, 249350]",...,2,56,192,248,"{'1.1': [[[47.88 11.6 ], [46.72 12.984], [48....","{'1.1': [[48.3, 13.7], [47.84, 13.695], [47.75...","{'1.1': 22.351771133170907, '1.2': 0.988053123...","{'1.1': 3.033430232558146, '1.2': 0.9709302325...",1.2,False
1,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[389, 405]",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,"[1.1, 1.2]",7977066,76318450,"[[1126742, 1326741], [3526740, 3726740], [5826...","[389747, 407142]",...,2,16,389,405,"{'1.1': [[[46.1 33.44], [43.97 34.94], [46.16...","{'1.1': [[46.16, 36.0], [46.12, 35.97], [46.16...","{'1.1': 0.8902350171923165, '1.2': 25.90464148...","{'1.1': 0.8771802325581461, '1.2': 6.564680232...",1.1,False
2,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[916, 929]",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,"[1.1, 1.2]",7977066,76318450,"[[1126742, 1326741], [3526740, 3726740], [5826...","[917544, 931741]",...,2,13,916,929,"{'1.1': [[[61.34 18. ], [58.6 18.38], [60.25...","{'1.1': [[60.25, 19.8], [60.78, 19.5], [60.78,...","{'1.1': 21.949273716111037, '1.2': 1.066196743...","{'1.1': 14.970930232558146, '1.2': 0.908430232...",1.2,False
3,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[929, 948]",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,"[1.1, 1.2]",7977066,76318450,"[[1126742, 1326741], [3526740, 3726740], [5826...","[932141, 950539]",...,2,19,929,948,"{'1.1': [[[59.7 15.125], [60.84 16.25], [61....","{'1.1': [[61.47, 16.03], [61.22, 15.4], [61.0,...","{'1.1': 25.589320821862895, '1.2': 0.630309449...","{'1.1': 16.189680232558146, '1.2': 0.345930232...",1.2,False
4,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[961, 983]",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,"[1.1, 1.2]",7977066,76318450,"[[1126742, 1326741], [3526740, 3726740], [5826...","[963342, 986354]",...,2,22,961,983,"{'1.1': [[[58.62 12.96], [60.28 14.03], [59.75...","{'1.1': [[59.75, 13.06], [59.88, 14.26], [59.9...","{'1.1': 26.99243314043086, '1.2': 0.6916250894...","{'1.1': 14.470930232558146, '1.2': 0.408430232...",1.2,False


## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

In [41]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_2"

In [42]:
NOSE_POKE_PKL = "{}_nose_poke_analysis.pkl".format(OUTPUT_PREFIX)

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [43]:
NOSE_POKE_DF.columns

Index(['session_dir', 'tone_frames', 'box_1_port_entry_frames', 'video_name',
       'session_path', 'all_subjects', 'first_timestamp', 'last_timestamp',
       'tone_timestamps', 'box_1_port_entry_timestamps', 'box_top_left',
       'box_top_right', 'reward_port', 'box_bottom_left', 'box_bottom_right',
       'nose_index', 'entry_length', 'box_1_port_entry_start',
       'box_1_port_entry_end', 'sliced_locations', 'nose_coordinates',
       'nose_to_reward_port_distance', 'nose_to_reward_port_distance_x',
       'entry_winner', 'contested'],
      dtype='object')

In [44]:
contested_df = NOSE_POKE_DF[NOSE_POKE_DF["contested"] == True]

In [45]:
NOSE_POKE_DF["sliced_locations"].iloc[0]["1.1"].shape

(56, 6, 2)

In [46]:
NOSE_POKE_DF["entry_length"].iloc[0]

56

In [47]:
contested_df.head()

,session_dir,tone_frames,box_1_port_entry_frames,video_name,session_path,all_subjects,first_timestamp,last_timestamp,tone_timestamps,box_1_port_entry_timestamps,...,nose_index,entry_length,box_1_port_entry_start,box_1_port_entry_end,sliced_locations,nose_coordinates,nose_to_reward_port_distance,nose_to_reward_port_distance_x,entry_winner,contested
155,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[9720, 9868]",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,"[1.1, 1.2]",7977066,76318450,"[[1126742, 1326741], [3526740, 3726740], [5826...","[9739343, 9887730]",...,2,148,9720,9868,"{'1.1': [[[40.5 34.94], [38.9 36.3], [44.44 3...","{'1.1': [[44.44, 36.4], [44.97, 36.7], [44.78,...","{'1.1': 1.0098364634611707, '1.2': 0.588110045...","{'1.1': 0.8415697674418539, '1.2': 0.185319767...",1.2,True
156,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[9870, 9899]",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,"[1.1, 1.2]",7977066,76318450,"[[1126742, 1326741], [3526740, 3726740], [5826...","[9888732, 9917332]",...,2,29,9870,9899,"{'1.1': [[[42.22 34.97], [40.6 37.1], [44.84 3...","{'1.1': [[44.84, 36.72], [44.75, 36.72], [44.6...","{'1.1': 0.9734128200563306, '1.2': 0.598293383...","{'1.1': 0.43531976744185386, '1.2': 0.28343023...",1.2,True
165,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[10516, 10518]",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,"[1.1, 1.2]",7977066,76318450,"[[1126742, 1326741], [3526740, 3726740], [5826...","[10536335, 10539137]",...,2,2,10516,10518,"{'1.1': [[[39.97 34.56], [38.9 35.6], [44.44 3...","{'1.1': [[44.44, 36.0], [43.9, 36.0]], '1.2': ...","{'1.1': 0.855168345366441, '1.2': 0.4728801936...","{'1.1': 0.8415697674418539, '1.2': 0.404069767...",1.2,True
167,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[10640, 10723]",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,"[1.1, 1.2]",7977066,76318450,"[[1126742, 1326741], [3526740, 3726740], [5826...","[10660132, 10744137]",...,2,83,10640,10723,"{'1.1': [[[42.28 34.6 ], [40.88 36.1 ], [44.8 ...","{'1.1': [[44.8, 36.44], [44.56, 36.5], [44.28,...","{'1.1': 0.7517168454547564, '1.2': 1.168323123...","{'1.1': 0.46656976744185386, '1.2': 0.77906976...",1.1,True
211,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[14368, 14606]",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,"[1.1, 1.2]",7977066,76318450,"[[1126742, 1326741], [3526740, 3726740], [5826...","[14396328, 14634333]",...,2,238,14368,14606,"{'1.1': [[[39.97 34.94], [38.2 36.34], [44.3 ...","{'1.1': [[44.3, 36.38], [44.2, 36.38], [44.03,...","{'1.1': 1.10085393736415, '1.2': 0.97668180138...","{'1.1': 0.9665697674418539, '1.2': 0.841569767...",1.2,True


In [48]:
user_df = contested_df[['video_name',
                        'all_subjects',
                        'box_1_port_entry_timestamps',
                        'entry_winner']].copy()

In [58]:
user_df.head()

,video_name,all_subjects,box_1_port_entry_timestamps,entry_winner
155,20230612_112630_standard_comp_to_training_D1_s...,"[1.1, 1.2]","[9739343, 9887730]",1.2
156,20230612_112630_standard_comp_to_training_D1_s...,"[1.1, 1.2]","[9888732, 9917332]",1.2
165,20230612_112630_standard_comp_to_training_D1_s...,"[1.1, 1.2]","[10536335, 10539137]",1.2
167,20230612_112630_standard_comp_to_training_D1_s...,"[1.1, 1.2]","[10660132, 10744137]",1.1
211,20230612_112630_standard_comp_to_training_D1_s...,"[1.1, 1.2]","[14396328, 14634333]",1.2


In [57]:
video_name = user_df['video_name'].iloc[0]
video_name = r"C:\Users\yongc\Desktop\Leo Code\\" +video_name + 'contested_frames.csv'

In [61]:
user_df.to_csv(video_name)
contested_df.to_csv(r"C:\Users\yongc\Desktop\Leo Code\contested.csv")